## simple Gen AI project

In [1]:
import os
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

#used for Langsmith tracking
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')

In [ ]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
loader.load()

In [ ]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
Documets=text_splitter.split_documents(loader.load())


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
vectore_store_db=FAISS.from_documents(Documets,embedding)
vectore_store_db

In [ ]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectore_store_db.similarity_search(query)
result[0].page_content

## Retrivel chain
- Retrieval Chain is a pattern used to build an LLM-powered Q&A system that can search and answer from your own data, like PDFs, websites, Notion docs, or databases.

In [ ]:
#Retrieval chain , Docment chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate(
    """Answer the following question based only on the provided context:
<context>
{context}
</context>

 """
 )
Documet_chain=create_stuff_documents_chain(
    llm,prompt
)

In [ ]:
from langchain_core.documents import Document
Documet_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [ ]:
### Input--->Retriever--->vectorstoredb

vectore_store_db

In [ ]:
retriver=vectore_store_db.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriver,Documet_chain)
retrieval_chain

In [ ]:
#get the responce from the llm
responce=retrieval_chain.invoke({'input':"LangSmith has two usage limits: total traces and extended"})
responce['answer']